In [1]:
import pandas as pd
import numpy as np
import os
import dotenv
from dotenv import load_dotenv
from common.PortConnect import Port_Connect

In [2]:
load_dotenv()
api_key = os.getenv("API_KEY")

fmp = Port_Connect(api_key=api_key)
apple = fmp.historical_closing_price('SYK')
apple



/Users/franciscoruiz/Desktop/APPLIED_FINANCE_ANALYTICS/APPLIED_FINANCE_ANALYTICS/common/PortConnect.py:56: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['date'] = pd.to_datetime(df['date'],infer_datetime_format=True)
/Users/franciscoruiz/Desktop/APPLIED_FINANCE_ANALYTICS/APPLIED_FINANCE_ANALYTICS/common/PortConnect.py:62: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df.set_index = pd.to_datetime(df.index, infer_datetime_format=True)


,symbol,close
date,,
1979-05-02,SYK,0.083335
1979-05-03,SYK,0.086421
1979-05-04,SYK,0.081792
1979-05-07,SYK,0.077162
1979-05-08,SYK,0.077162
...,...,...
2025-06-05,SYK,383.100000
2025-06-06,SYK,383.890000
2025-06-09,SYK,382.280000


## Calculate Key Metric Statistics for a holding

In [3]:
from common.Portfolio import Portfolio_Stats



In [4]:
port = Portfolio_Stats()

In [5]:
change = apple['close'].pct_change()
change

date
1979-05-02         NaN
1979-05-03    0.037031
1979-05-04   -0.053563
1979-05-07   -0.056607
1979-05-08    0.000000
                ...   
2025-06-05    0.001438
2025-06-06    0.002062
2025-06-09   -0.004194
2025-06-10    0.007246
2025-06-11   -0.010310
Name: close, Length: 11624, dtype: float64

In [6]:
type(change)

pandas.core.series.Series

In [7]:
filtered_change = change.loc['2010-01-01':'2025-12-31']

In [8]:
port.annualize_rets(filtered_change,252)

0.14030436770929877

In [9]:
port.annualize_vol(filtered_change,252)

0.24227408136647288

In [11]:
max_duration = port.drawdown_duration_from_returns(filtered_change)
max_duration

724

In [14]:
tickers = ['SYK','ISRG','BLK','BSX','HD',"VOO"]

prices = fmp.get_closing_prices(tickers,from_date='2010-01-01')
prices

/Users/franciscoruiz/Desktop/APPLIED_FINANCE_ANALYTICS/APPLIED_FINANCE_ANALYTICS/common/PortConnect.py:56: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['date'] = pd.to_datetime(df['date'],infer_datetime_format=True)
/Users/franciscoruiz/Desktop/APPLIED_FINANCE_ANALYTICS/APPLIED_FINANCE_ANALYTICS/common/PortConnect.py:62: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df.set_index = pd.to_datetime(df.index, infer_datetime_format=True)
/Users/franciscoruiz/Desktop/APPLIED_FINANCE_ANALYTICS/APPLIED_FINANCE_ANALYTICS/common/PortConnect.py:56: UserWarning: T

symbol,BLK,BSX,HD,ISRG,SYK,VOO
date,,,,,,
2010-01-04,238.58,9.01,28.67,34.31,52.16,NaN
2010-01-05,239.61,9.04,28.88,34.15,52.65,NaN
2010-01-06,234.67,9.16,28.78,34.65,53.46,NaN
2010-01-07,237.25,9.09,29.12,34.52,55.25,NaN
2010-01-08,238.92,9.00,28.98,34.33,55.42,NaN
...,...,...,...,...,...,...
2025-06-05,985.46,102.83,369.28,558.06,383.10,545.09
2025-06-06,989.05,102.09,367.33,557.08,383.89,550.66
2025-06-09,988.84,100.04,365.20,526.15,382.28,551.25


In [15]:
dfs = []
for ticker in prices.columns:
    change = prices[ticker].pct_change()
    summary = port.summary_stats(change)
    dfs.append(summary)


pd.concat(dfs)


,Annualized Return,Annualized Vol,Annualized Semideviation,Sharpe Ratio,Skewness,Kurtosis,Cornish-Fisher VaR (5%),Historic CVaR (5%),Max Drawdown,Drawdown Duration (Days)
BLK,0.096960,0.276937,0.200320,0.324583,0.051173,9.896530,0.025490,0.040000,-0.453314,1129
BSX,0.168556,0.283885,0.208833,0.644248,-0.136319,9.270553,0.027056,0.041629,-0.483368,1242
HD,0.178508,0.233047,0.178075,0.809803,-0.618193,17.432743,0.021582,0.033392,-0.384058,1043
ISRG,0.193241,0.320771,0.234587,0.675693,0.267017,12.654900,0.026830,0.045976,-0.499042,1410
SYK,0.137724,0.242142,0.184020,0.568435,-0.109149,13.390085,0.021730,0.035554,-0.438028,727
VOO,0.116381,0.173190,0.141079,0.604636,-0.377650,16.440593,0.015609,0.026421,-0.343014,745


In [16]:
port.summary_stats(filtered_change)

,Annualized Return,Annualized Vol,Annualized Semideviation,Sharpe Ratio,Skewness,Kurtosis,Cornish-Fisher VaR (5%),Historic CVaR (5%),Max Drawdown,Drawdown Duration (Days)
close,0.140304,0.242274,0.18402,0.582006,-0.107628,13.364778,0.021734,0.035554,-0.438028,724


In [ ]:
port.cvar_historic(filtered_change)

0.03555434142407394

In [ ]:
port.drawdown_duration_from_returns(filtered_change).max().days

724

In [ ]:
port.drawdown(filtered_change)

,Wealth,Previous Peak,Drawdown
date,,,
2010-01-04,1035.537026,1035.537026,0.000000
2010-01-05,1045.265039,1045.265039,0.000000
2010-01-06,1061.346039,1061.346039,0.000000
2010-01-07,1096.883065,1096.883065,0.000000
2010-01-08,1100.258090,1100.258090,0.000000
...,...,...,...
2025-06-05,7605.717689,7939.249553,-0.042011
2025-06-06,7621.401628,7939.249553,-0.040035
2025-06-09,7589.438158,7939.249553,-0.044061


In [ ]:
filtered_change

date
2010-01-04    0.035537
2010-01-05    0.009394
2010-01-06    0.015385
2010-01-07    0.033483
2010-01-08    0.003077
                ...   
2025-06-05    0.001438
2025-06-06    0.002062
2025-06-09   -0.004194
2025-06-10    0.007246
2025-06-11   -0.010310
Name: close, Length: 3884, dtype: float64